<h2>미국 항공편 예제

In [16]:
from pyspark.sql import SparkSession


schema = "'date' STRING, 'delay' INT, 'distance' INT, 'origin' STRING, 'destination' STRING"
spark = (SparkSession
         .builder
         .appName('SparkSQLExampleApp')
         .getOrCreate())

filepath = 'departuredelays.csv'

df = (spark.read.format('csv')
      .option('inferSchema','true') #데이터의 타입(숫자, 문자열 등)을 자동으로 추론하도록 설정합니다.
      .option('header','true') 
      .load(filepath))

df.createOrReplaceTempView('us_delay_flights_tbl') #임시뷰를 만드는 함수, 임시뷰를 만들면 SQL문이 사용가능하다다

메타데이터 보기

In [14]:
spark.catalog.listDatabases()
spark.catalog.listTables()
spark.catalog.listColumns('us_delay_flights_tbl')

[Column(name='date', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='delay', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='distance', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='origin', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='destination', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False)]

Distance가 1000이상인 것 조회   

In [ ]:
spark.sql("""SELECT distance, origin, destination
          FROM us_delay_flights_tbl
          WHERE distance > 1000
          ORDER BY distance DESC """).show(5)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 5 rows



SFO(샌프란시스코)와 ORD(시카고)간 2시간 이상 지연이 있었던 항공편 찾기

In [22]:
spark.sql("""SELECT date, delay, origin, destination
          FROM us_delay_flights_tbl
          WHERE delay > 120 and origin == 'SFO' and destination == 'ORD'
          ORDER BY delay DESC""").show(5)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|2190925| 1638|   SFO|        ORD|
|1031755|  396|   SFO|        ORD|
|1022330|  326|   SFO|        ORD|
|1051205|  320|   SFO|        ORD|
|1190925|  297|   SFO|        ORD|
+-------+-----+------+-----------+
only showing top 5 rows



In [25]:
spark.sql("""SELECT delay, origin, destination,
          CASE
            WHEN delay > 360 then 'very long delays'
            WHEN delay >= 120 and delay <= 360 then 'long delays'
            WHEN delay >= 60 and delay < 120 then 'short delays'
            WHEN delay > 0 and delay < 60 then 'Tolerable delays'
            WHEN delay = 0 then 'No delays'
            ELSE 'Early'
          END AS Flight_Delays
          FROM us_delay_flights_tbl
          ORDER BY origin, delay DESC""").show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  long delays|
|  305|   ABE|        ATL|  long delays|
|  275|   ABE|        ATL|  long delays|
|  257|   ABE|        ATL|  long delays|
|  247|   ABE|        ATL|  long delays|
|  247|   ABE|        DTW|  long delays|
|  219|   ABE|        ORD|  long delays|
|  211|   ABE|        ATL|  long delays|
|  197|   ABE|        DTW|  long delays|
|  192|   ABE|        ORD|  long delays|
+-----+------+-----------+-------------+
only showing top 10 rows



임시 뷰가 아닌 파이썬 데이터프레임 표현식

In [26]:
from pyspark.sql.functions import col, desc
(df
 .select('distance','origin','destination')
 .where(col('distance')>1000)
 .orderBy(desc('distance'))
).show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



<h3> SQL 데이터베이스와 테이블 생성하기

In [ ]:
#데이터베이스 생성
spark.sql("CREATE DATABASE learn_spark_db")
spark.sql("USE learn_spark_db")

DataFrame[]

생성한 데이터베이스 안에 테이블을 생성하기

In [ ]:
schema = "date STRING, delay INT, distance INT, origin STRING, destination STRING"
flights_df = spark.read.csv(filepath, header=True, schema=schema)
flights_df.write.saveAsTable('managed_us_delay_flights_tbl')

항공사 데이터에서 전역/일반 임시 뷰 생성

In [36]:

df_sfo = spark.sql("""SELECT date, delay, origin, destination
                      FROM us_delay_flights_tbl
                      WHERE origin = 'SFO'""")
df_jfk = spark.sql("""SELECT date, delay, origin, destination
                      FROM us_delay_flights_tbl
                      WHERE origin = 'JFK'""")

# create a temporary and global temporary view
df_sfo.createOrReplaceGlobalTempView("us_origin_airport_SFO_global_tmp_view")
df_jfk.createOrReplaceTempView("us_origin_airport_JFK_tmp_view")

In [ ]:
# 전역 임시 뷰 접근
# 방법1
spark.read.table("global_temp.us_origin_airport_SFO_global_tmp_view").show(5)
# 방법2
spark.sql("SELECT * FROM global_temp.us_origin_airport_SFO_global_tmp_view").show(5)

In [ ]:

# 일반 임시 뷰 접근
# 방법1
spark.read.table("us_origin_airport_JFK_tmp_view").show(5)
# 방법2
spark.sql("SELECT * FROM us_origin_airport_JFK_tmp_view").show(5)

In [ ]:
# 뷰 드롭
spark.catalog.dropGlobalTempView("us_origin_airport_SFO_global_tmp_view")
spark.catalog.dropTempView("us_origin_airport_JFK_tmp_view")

파키에 파일을 데이터프레임으로 불러오기

In [18]:
filepath = 'part-r-00000-1a9822ba-b8fb-4d8e-844a-ea30d0801b9e.gz.parquet'
df=spark.read.format('parquet').load(filepath)
df.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

JSON 파일을 데이터프레임으로 읽기

In [20]:
filepath = '2015-summary.json'
df = spark.read.format('json').load(filepath)
df.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

데이터 프레임을 json파일로 쓰기

In [ ]:
j = (df.write.format('json')
 .mode('overwrite')
 .option('compression','snappy')
 .save('저장 경로')
)

In [24]:
spark.stop()